In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import joblib
# Load the digits dataset
digits = datasets.load_digits()

In [ ]:
digits = datasets.load_digits()

_, axes = plt.subplots(nrows=1, ncols=10, figsize=(10, 3))
for ax, image, label in zip(axes, digits.images, digits.target):
    ax.set_axis_off()
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation="nearest")
    ax.set_title("Train: %i" % label)

In [ ]:
digits = datasets.load_digits()
digits.data = digits.data/np.max(digits.data)
# Split data into 50% train and 50% test subsets
X_train, X_test, y_train, y_test = train_test_split(
    digits.data, digits.target, test_size=1/5, shuffle=True, random_state=42
)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Create a support vector classifier
svc = SVC(probability=True)

# Define the hyperparameters and their values to experiment with
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['rbf', 'poly'],
    'degree': [1, 2, 3],
    'gamma': ['scale', 'auto', 1, 0.1, 0.01, 0.001]

}
# Perform grid search to find the best combination of hyperparameter values
grid_search_svm = GridSearchCV(svc, param_grid, cv=5)
grid_search_svm.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search_svm.best_params_
print("Best hyperparameters:", best_params)
print("Best score:", grid_search_svm.best_score_)


In [ ]:
from xgboost import XGBClassifier
# Define parameter grid
param_grid = {
    'max_depth': [3,5,7],
    'learning_rate': [0.1, 0.01, 0.05],
    'n_estimators': [100, 300, 500],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# Create XGBClassifier
model = XGBClassifier(objective='multi:softmax', random_state=42, n_jobs=1)

# Perform grid search
grid_search_xgb = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search_xgb.fit(X_train, y_train)

# Print best score and parameters
print(f"Best score: {grid_search_xgb.best_score_:.3f}")
print(f"Best parameters: {grid_search_xgb.best_params_}")

In [ ]:
# Use the best model for prediction
best_model = grid_search_svm.best_estimator_

# Make predictions on the test set
predictions = best_model.predict(X_test)

# Print the classification report
print("Classification report for classifier %s:\n%s\n" % (best_model, metrics.classification_report(y_test, predictions)))


In [ ]:
from itables import show
predictions_proba = best_model.predict_proba(X_test)
#convert predictions_proba to a dataframe with predictions using max
predictions = pd.DataFrame(predictions_proba).idxmax(axis=1).values
#get the second prediction probability for each prediction
show(predictions_proba)

In [ ]:
best_model = best_model.fit(digits.data, digits.target)

In [ ]:
joblib.dump(best_model, 'model_svm.pkl')